In [1]:
import pandas as pd
import os


In [2]:
def calculate_data_stats(dataframe, filename):
    # Set 'Timestamp' as index if present
    if 'Timestamp' in dataframe.columns:
        dataframe['Timestamp'] = pd.to_datetime(dataframe['Timestamp'], errors='coerce')
        dataframe = dataframe.set_index('Timestamp')
    
    stats = {}
    stats['filename'] = filename
    
    stats["Years"] = f"{dataframe.index[0]} to {dataframe.index[-1]}"
    
    stats["Rows"] = dataframe.shape[0]
    stats["Features"] = dataframe.shape[1]
    stats["Empty Rows"] = dataframe.isnull().all(axis=1).sum()
    
    missing_cols = []
    present_cols = []
    for p in dataframe.columns:
        count = dataframe[p].notnull().sum()
        percent = round(count * 100 / dataframe.shape[0], 2)
        stats[p] = percent
        if count == 0:
            missing_cols.append(p)
        else:
            present_cols.append(p)
    
    stats["Missing Features"] = str(missing_cols)
    stats["Rows with Missing Values"] = dataframe[present_cols].isnull().any(axis=1).sum() if present_cols else 0
    
    # Count ADDITIONAL missing (only from present columns)
    if present_cols:
        missing_counts = dataframe[present_cols].isnull().sum(axis=1)
        missing_bins = round((missing_counts.value_counts() * 100 / dataframe.shape[0]), 2).sort_index()
        for bin_count, percent in missing_bins.items():
            stats[f'additional_missing_{int(bin_count)}'] = percent
    
    return stats

In [3]:
def batch_data_stats(folder_path, stats_fn):
    """
    Runs the given stats function on every CSV file in the folder,
    appends the overall stats (scalars only) to a list, and returns a DataFrame.
    """
    results = []
    for fname in sorted(os.listdir(folder_path)):
        if fname.endswith('.csv'):
            fpath = os.path.join(folder_path, fname)
            try:
                df = pd.read_csv(fpath)
                stats = stats_fn(df,fname)
                results.append(stats)
            except Exception as e:
                print(f"Error processing {fname}: {e}")
    return pd.DataFrame(results)

# Example usage:
summary_df = batch_data_stats('/home/rishi/ML Projects/Air Pollution/sites_comb', calculate_data_stats)
summary_df.head()

,filename,Years,Rows,Features,Empty Rows,PM2.5 (µg/m³),PM10 (µg/m³),NO (µg/m³),NO2 (µg/m³),NOx (ppb),...,additional_missing_14,additional_missing_15,additional_missing_17,additional_missing_16,additional_missing_18,additional_missing_19,additional_missing_20,additional_missing_21,additional_missing_22,additional_missing_23
0,site_103_CRRI_Mathura_Road_Delhi_IMD_15Min.csv,2017-01-01 00:00:00 to 2025-12-31 23:00:00,78888,24,0,86.38,84.54,82.70,85.92,86.85,...,0.04,0.01,11.58,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,site_104_Burari_Crossing_Delhi_IMD_15Min.csv,2017-01-01 00:00:00 to 2025-12-31 23:00:00,78888,24,0,59.27,59.52,48.77,53.84,60.00,...,35.43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,site_105_North_Campus_DU_Delhi_IMD_15Min.csv,2017-01-01 00:00:00 to 2025-12-31 23:00:00,78888,24,0,84.39,84.11,82.20,83.29,86.39,...,0.02,0.00,12.87,0.01,NaN,NaN,NaN,NaN,NaN,NaN
3,site_106_IGI_Airport_(T3)_Delhi_IMD_15Min.csv,2017-01-01 00:00:00 to 2025-12-31 23:00:00,78888,24,0,83.73,84.43,81.77,81.23,81.49,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,site_107_Pusa_Delhi_IMD_15Min.csv,2017-01-01 00:00:00 to 2025-12-31 23:00:00,78888,24,0,82.25,83.06,82.91,83.87,84.91,...,0.11,0.02,0.01,0.03,14.33,NaN,NaN,NaN,NaN,NaN


In [4]:
summary_df.to_csv("summary_df.csv")

In [5]:
(summary_df==0).sum(axis=0)

filename                      0
Years                         0
Rows                          0
Features                      0
Empty Rows                  363
PM2.5 (µg/m³)                 3
PM10 (µg/m³)                  8
NO (µg/m³)                    7
NO2 (µg/m³)                   2
NOx (ppb)                     6
NH3 (µg/m³)                  63
SO2 (µg/m³)                  23
CO (mg/m³)                    6
Ozone (µg/m³)                21
Benzene (µg/m³)              65
Toluene (µg/m³)             153
Xylene (µg/m³)              397
O Xylene (µg/m³)            525
Eth-Benzene (µg/m³)         165
MP-Xylene (µg/m³)           171
AT (°C)                     101
RH (%)                       42
WS (m/s)                     41
WD (deg)                     39
RF (mm)                     141
TOT-RF (mm)                  83
SR (W/mt2)                   95
BP (mmHg)                   297
VWS (m/s)                   486
Missing Features              0
Rows with Missing Values      0
addition

In [18]:
features=["PM2.5 (µg/m³)", "PM10 (µg/m³)", "NO2 (µg/m³)","NH3 (µg/m³)","SO2 (µg/m³)","CO (mg/m³)","Ozone (µg/m³)", "AT (°C)", "RH (%)"]

for feature in features:
    df = pd.read_csv(f"/home/rishi/ML Projects/Air Pollution/CPCB/visualize_max/{feature.replace('/', '_').replace(' ', '_')}_df_max_cpcb_new_limits.csv")
    
    # Calculate missing percentage for each column (site)
    missing_pct = df.isnull().sum(axis=0) * 100 / len(df)
    
    # Count sites with missing data less than each threshold
    thresholds = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
    
    print(f"\n{feature}:")
    for threshold in thresholds:
        count = (missing_pct <= threshold).sum()
        print(f"  <={threshold}%: {count} sites")
    print(f"Total sites: {len(missing_pct)}")


PM2.5 (µg/m³):
  <=10%: 185 sites
  <=20%: 323 sites
  <=30%: 416 sites
  <=40%: 462 sites
  <=50%: 483 sites
  <=60%: 505 sites
  <=70%: 526 sites
  <=80%: 539 sites
  <=90%: 544 sites
  <=100%: 565 sites
Total sites: 565

PM10 (µg/m³):
  <=10%: 187 sites
  <=20%: 327 sites
  <=30%: 417 sites
  <=40%: 460 sites
  <=50%: 481 sites
  <=60%: 504 sites
  <=70%: 519 sites
  <=80%: 532 sites
  <=90%: 539 sites
  <=100%: 565 sites
Total sites: 565

NO2 (µg/m³):
  <=10%: 217 sites
  <=20%: 334 sites
  <=30%: 424 sites
  <=40%: 468 sites
  <=50%: 491 sites
  <=60%: 513 sites
  <=70%: 534 sites
  <=80%: 542 sites
  <=90%: 548 sites
  <=100%: 565 sites
Total sites: 565

NH3 (µg/m³):
  <=10%: 194 sites
  <=20%: 300 sites
  <=30%: 384 sites
  <=40%: 426 sites
  <=50%: 445 sites
  <=60%: 462 sites
  <=70%: 480 sites
  <=80%: 489 sites
  <=90%: 492 sites
  <=100%: 565 sites
Total sites: 565

SO2 (µg/m³):
  <=10%: 205 sites
  <=20%: 331 sites
  <=30%: 410 sites
  <=40%: 457 sites
  <=50%: 478 sites


In [20]:
# Get sites with <20% missing data for each pollutant and find intersection
pollutants = ["PM2.5 (µg/m³)", "PM10 (µg/m³)", "NO2 (µg/m³)", "NH3 (µg/m³)", "SO2 (µg/m³)", "CO (mg/m³)", "Ozone (µg/m³)"]
threshold = 30  # 20% missing data threshold

sites_by_pollutant = {}

for pollutant in pollutants:
    df = pd.read_csv(f"/home/rishi/ML Projects/Air Pollution/CPCB/visualize_median_2023/{pollutant.replace('/', '_').replace(' ', '_')}_df_cpcb_new_limits.csv")
    
    # Calculate missing percentage for each column (site)
    missing_pct = df.isnull().sum(axis=0) * 100 / len(df)
    
    # Get sites with <20% missing
    good_sites = missing_pct[missing_pct < threshold].index.tolist()
    sites_by_pollutant[pollutant] = set(good_sites)
    
    print(f"{pollutant}: {len(good_sites)} sites with <{threshold}% missing")

# Find intersection of all pollutants
final_sites = set.intersection(*sites_by_pollutant.values())

print(f"\n{'='*60}")
print(f"Sites with <{threshold}% missing data for ALL pollutants: {len(final_sites)}")
print(f"{'='*60}")
print(f"\nFinal site list:")
print(sorted(final_sites))

PM2.5 (µg/m³): 416 sites with <30% missing
PM10 (µg/m³): 417 sites with <30% missing
NO2 (µg/m³): 424 sites with <30% missing
NH3 (µg/m³): 384 sites with <30% missing
SO2 (µg/m³): 410 sites with <30% missing
CO (mg/m³): 431 sites with <30% missing
Ozone (µg/m³): 413 sites with <30% missing

Sites with <30% missing data for ALL pollutants: 340

Final site list:
['Unnamed: 0', 'site_113_Shadipur_Delhi_CPCB_15Min.csv', 'site_114_IHBAS_Dilshad_Garden_Delhi_CPCB_15Min.csv', 'site_115_NSIT_Dwarka_Delhi_CPCB_15Min.csv', 'site_117_ITO_Delhi_CPCB_15Min.csv', 'site_118_DTU_Delhi_CPCB_15Min.csv', 'site_119_Sirifort_Delhi_CPCB_15Min.csv', 'site_122_Mandir_Marg_Delhi_DPCC_15Min.csv', 'site_124_R_K_Puram_Delhi_DPCC_15Min.csv', 'site_125_Punjabi_Bagh_Delhi_DPCC_15Min.csv', 'site_134_Police_Commissionerate_Jaipur_RSPCB_15Min.csv', 'site_1390_Moti_Doongri_Alwar_RSPCB_15Min.csv', 'site_1391_RIICO_Ind._Area_III_Bhiwadi_RSPCB_15Min.csv', 'site_1392_Civil_Lines__Ajmer_RSPCB_15Min.csv', 'site_1393_Adarsh_Na